- 所有的graph都是按照GraphTruple 实例作为输入，并返回以GraphTruple作为输出。

- 图跟图之间全局属性，节点属性和边属性的维度要一样的
- 将字典类型的数据通过\[\]进行拼接，然后使用utils_np.data_dicts_to_graph_truple将信息转化为truple.该函数有以下几点作用：
    - 处理节点和边的函数可以并行。
    - 能够计算每个图的节点数和边数，并将其放在n_node和n_edge中，这样他的长度就等于图的数量，而且这样有利于进行分割，让全局的属性广播到属于每一个节点和边。
    - 将节点和边进行拼接，这样索引就会出现偏移。

- 进行简单的图的可视化graphs_tuple是经过data_dicts_to_graph_truple处理过的对象

In [ ]:
# 可视化图
import networkx as nx
graphs_nx = utils_np.graphs_tuple_to_networkxs(graphs_tuple)
_, axs = plt.subplots(ncols=2, figsize=(6, 3))
for iax, (graph_nx, ax) in enumerate(zip(graphs_nx, axs)):
  nx.draw(graph_nx, ax=ax)
  ax.set_title("Graph {}".format(iax))
# 打印图的一些信息
def print_graphs_tuple(graphs_tuple):
  print("Shapes of `GraphsTuple`'s fields:")
  print(graphs_tuple.map(lambda x: x if x is None else x.shape, fields=graphs.ALL_FIELDS))
  print("\nData contained in `GraphsTuple`'s fields:")
  print("globals:\n{}".format(graphs_tuple.globals))
  print("nodes:\n{}".format(graphs_tuple.nodes))
  print("edges:\n{}".format(graphs_tuple.edges))
  print("senders:\n{}".format(graphs_tuple.senders))
  print("receivers:\n{}".format(graphs_tuple.receivers))
  print("n_node:\n{}".format(graphs_tuple.n_node))
  print("n_edge:\n{}".format(graphs_tuple.n_edge))

print_graphs_tuple(graphs_tuple)
### networkx 中建立一张图

graph_nx = nx.OrderedMultiDiGraph()
# Globals.
graph_nx.graph["features"] = np.array([0.6, 0.7, 0.8])

# Nodes.
graph_nx.add_node(0, features=np.array([0.3, 1.3]))
graph_nx.add_node(1, features=np.array([0.4, 1.4]))
graph_nx.add_node(2, features=np.array([0.5, 1.5]))
graph_nx.add_node(3, features=np.array([0.6, 1.6]))
# Edges.
graph_nx.add_edge(0, 1, features=np.array([3.6, 3.7]))
graph_nx.add_edge(2, 0, features=np.array([5.6, 5.7]))
graph_nx.add_edge(3, 0, features=np.array([6.6, 6.7]))

ax = plt.figure(figsize=(3, 3)).gca()
nx.draw(graph_nx, ax=ax)
ax.set_title("Graph")

graphs_tuple = utils_np.networkxs_to_graphs_tuple([graph_nx])

print_graphs_tuple(graphs_tuple)